In [ ]:
# 1. 라이브러리 임포트
from langgraph.graph import StateGraph, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import tools_condition

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda

from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions

from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

from langchain_openai import ChatOpenAI
import os

# 2. 상태 정의
class AgentState(MessagesState):
    pass

# 3. 벡터 DB 로드 (bge-m3 기준)
embeddings = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/menu_db", embeddings,
    allow_dangerous_deserialization=True
)

# 4. 도구 정의
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴 정보 검색 도구입니다. 예: 아메리카노 가격, 디저트 종류 등"""
    docs = menu_db.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in docs])

tools = [search_cafe_menu]

# 5. LLM 정의 (OpenAI)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 6. ReAct Agent 구성
agent_runnable = (
    RunnableLambda(lambda x: x["messages"]) |
    format_to_openai_functions |
    llm.bind_tools(tools) |
    ReActSingleInputOutputParser()
)

# 7. 노드 정의
def agent_node(state: AgentState) -> AgentState:
    response = agent_runnable.invoke(state)
    return {"messages": state["messages"] + [response]}

def tool_node(state: AgentState) -> AgentState:
    tool_call = state["messages"][-1].tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    # 도구 실행
    result = search_cafe_menu.invoke(tool_args["query"])
    tool_msg = AIMessage(content=result, tool_call_id=tool_call["id"])

    return {"messages": state["messages"] + [tool_msg]}

# 8. LangGraph 정의
builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_node("tool", tool_node)

builder.set_entry_point("agent")
builder.add_conditional_edges("agent", tools_condition)  # 조건 분기: tool 호출 여부 자동 판단
builder.add_edge("tool", "agent")  # 순환 구조
builder.add_edge("agent", END)     # 도구 없이 종료 가능

graph = builder.compile()

# 9. 테스트 실행
response = graph.invoke({
    "messages": [HumanMessage(content="아메리카노와 아이스 아메리카노의 차이점과 가격 알려줘")]
})

# 10. 출력 확인
for msg in response["messages"]:
    print(f"[{msg.type}] {msg.content}")
